In [1]:
from dimod import (
    Binary,
    BinaryQuadraticModel,
    ConstrainedQuadraticModel,
)

from dwave.system import LeapHybridCQMSampler

import random
import pandas as pd
import numpy as np

# Define Token here to run
from credentials import TOKEN

from process_output import get_routes_from_sample, get_cost_routes, report_output, check_feasibility_sample, lazy_sanity_check

from vrp import *

In [ ]:
# Create a sample problem
num_destinations = 5
num_vehicles = 1
max_distance = 300

# Generate a random symmetric cost matrix
cost_matrix = [[0]*(num_destinations+1) for _ in range(num_destinations + 1)]

for i in range(num_destinations + 1):
    for j in range(i, num_destinations + 1):
        # Select random values that do not violate the triangle inequality
        if i == j:
            cost_matrix[i][j] = 0
        else:
            val = random.randint(5,9)
            cost_matrix[i][j] = val
            cost_matrix[j][i] = val

# Print the adjacency matrix
for row in cost_matrix:
    print(row)

In [ ]:
model = DefaultRoutingModel(num_destinations, distances=cost_matrix, num_vehicles=num_vehicles, max_distance = max_distance)

In [ ]:
model.build_constrained_model()
print(model.cqm)

In [ ]:
feasible_sampleset = model.run_constrained_model(TOKEN)

In [ ]:
lowest_energy_sample = feasible_sampleset.lowest().first.sample

routes = get_routes_from_sample(lowest_energy_sample, num_vehicles, num_destinations)

print(f'Best route: {routes}')

print(f'Best cost: {feasible_sampleset.lowest().first.energy}')

print(f'Cost calculated: {get_cost_routes(routes, distances=cost_matrix)}')

print(f'Feasible: {check_feasibility_sample(lowest_energy_sample, num_vehicles, num_destinations, cost_matrix, max_distance, debug=True)}')

# Sanity check on cost
assert feasible_sampleset.lowest().first.energy == get_cost_routes(routes, distances=cost_matrix) * 1.0, "Cost calculations do not agree"

In [ ]:
# Lazy sanity check
r = []
for i in range(1, 6):
    r.append(i)

min = 300
best_route = r
# Randomly sample a lot of options
for i in range(10000):
    random.shuffle(r)

    cost = get_cost_routes([r], cost_matrix)
    if cost < min:
        min = cost
        best_routes = [r]
print(min)
print(best_routes)

### Test 2: Capacitated Multiple Vehicle Routing Problem

In [ ]:
model2 = DefaultRoutingModel(num_destinations, cost_matrix, num_vehicles=2, max_distance=30)

In [ ]:
model2.build_constrained_model()

In [ ]:
feasible_sampleset2 = model2.run_constrained_model(token=TOKEN)

In [ ]:
def get_cost_routes(paths, distances):
    cost = 0

    num_vertices = len(distances)

    for path in paths:
        if len(path) > 0:

            # Cost in middle of path
            for i in range(len(path) - 1):
                cost += distances[path[i]][path[i+1]]

            # Cost of first choice
            cost += distances[0][path[0]]

            # Cost of last choice
            cost += distances[path[len(path) - 1]][0]

    return cost * 1.0

In [ ]:
lowest_energy_sample2 = feasible_sampleset2.lowest().first.sample

routes2 = get_routes_from_sample(lowest_energy_sample2, num_vehicles=2, num_steps=num_destinations)

print(f'Routes: {routes2}')

print(f'Best cost: {feasible_sampleset2.lowest().first.energy}')

In [ ]:
# Lazy sanity check
r = []
for i in range(1, 5+1):
    r.append(i)

min = 300
best_route = r
# Randomly sample a lot of options
for i in range(10000):
    random.shuffle(r)

    r1 = r[:2]
    r2 = r[2:]

    cost = get_cost_routes([r1, r2], cost_matrix)
    if cost < min:
        min = cost
        best_routes = [r1, r2]
print(min)
print(best_routes)

In [ ]:
val, route = lazy_sanity_check(num_destinations, distances=cost_matrix, num_vehicles=2)
print(val)
print(route)

### Larger Test

Try to generate a minimal example on which the solver fails

In [5]:
# Create a sample problem
num_destinations = 15
num_vehicles = 3
max_distance = 60

# Generate a random symmetric cost matrix
cost_matrix = [[0]*(num_destinations+1) for _ in range(num_destinations + 1)]

for i in range(num_destinations + 1):
    for j in range(i, num_destinations + 1):
        # Select random values that do not violate the triangle inequality
        if i == j:
            cost_matrix[i][j] = 0
        else:
            val = random.randint(5,9)
            cost_matrix[i][j] = val
            cost_matrix[j][i] = val

# Print the adjacency matrix
for row in cost_matrix:
    print(row)

[0, 5, 8, 9, 5, 5, 8, 5, 7, 7, 9, 7, 8, 7, 8, 9]
[5, 0, 9, 6, 8, 9, 6, 6, 9, 9, 5, 7, 8, 7, 5, 8]
[8, 9, 0, 9, 5, 9, 7, 5, 8, 6, 6, 7, 7, 6, 6, 5]
[9, 6, 9, 0, 6, 8, 9, 9, 7, 8, 7, 6, 8, 7, 8, 9]
[5, 8, 5, 6, 0, 9, 9, 7, 6, 6, 8, 7, 9, 9, 5, 8]
[5, 9, 9, 8, 9, 0, 6, 9, 6, 8, 6, 8, 8, 9, 9, 5]
[8, 6, 7, 9, 9, 6, 0, 8, 9, 6, 8, 8, 6, 9, 5, 6]
[5, 6, 5, 9, 7, 9, 8, 0, 7, 9, 9, 5, 8, 5, 9, 5]
[7, 9, 8, 7, 6, 6, 9, 7, 0, 8, 8, 7, 8, 5, 7, 6]
[7, 9, 6, 8, 6, 8, 6, 9, 8, 0, 8, 5, 7, 5, 9, 5]
[9, 5, 6, 7, 8, 6, 8, 9, 8, 8, 0, 6, 7, 5, 8, 7]
[7, 7, 7, 6, 7, 8, 8, 5, 7, 5, 6, 0, 8, 7, 8, 7]
[8, 8, 7, 8, 9, 8, 6, 8, 8, 7, 7, 8, 0, 6, 9, 5]
[7, 7, 6, 7, 9, 9, 9, 5, 5, 5, 5, 7, 6, 0, 5, 7]
[8, 5, 6, 8, 5, 9, 5, 9, 7, 9, 8, 8, 9, 5, 0, 6]
[9, 8, 5, 9, 8, 5, 6, 5, 6, 5, 7, 7, 5, 7, 6, 0]


In [12]:
model3 = DefaultRoutingModel(num_destinations, cost_matrix, num_vehicles=num_vehicles, max_distance=max_distance)
model3.build_constrained_model()
feasible_sampleset3 = model3.run_constrained_model(token=TOKEN)


Feasible solution found.



In [14]:
print(model3.cqm)

Constrained quadratic model: 720 variables, 63 constraints, 24339 biases

Objective
  5*Binary('0_1_0') + 8*Binary('0_2_0') + 9*Binary('0_3_0') + 5*Binary('0_4_0') + 5*Binary('0_5_0') + 8*Binary('0_6_0') + 5*Binary('0_7_0') + 7*Binary('0_8_0') + 7*Binary('0_9_0') + 9*Binary('0_10_0') + 7*Binary('0_11_0') + 8*Binary('0_12_0') + 7*Binary('0_13_0') + 8*Binary('0_14_0') + 9*Binary('0_15_0') + 5*Binary('1_1_0') + 8*Binary('1_2_0') + 9*Binary('1_3_0') + 5*Binary('1_4_0') + 5*Binary('1_5_0') + 8*Binary('1_6_0') + 5*Binary('1_7_0') + 7*Binary('1_8_0') + 7*Binary('1_9_0') + 9*Binary('1_10_0') + 7*Binary('1_11_0') + 8*Binary('1_12_0') + 7*Binary('1_13_0') + 8*Binary('1_14_0') + 9*Binary('1_15_0') + 5*Binary('2_1_0') + 8*Binary('2_2_0') + 9*Binary('2_3_0') + 5*Binary('2_4_0') + 5*Binary('2_5_0') + 8*Binary('2_6_0') + 5*Binary('2_7_0') + 7*Binary('2_8_0') + 7*Binary('2_9_0') + 9*Binary('2_10_0') + 7*Binary('2_11_0') + 8*Binary('2_12_0') + 7*Binary('2_13_0') + 8*Binary('2_14_0') + 9*Binary('2_15_0'

In [13]:
lowest_energy_sample3 = feasible_sampleset3.lowest().first.sample

routes3 = get_routes_from_sample(lowest_energy_sample3, num_vehicles=num_vehicles, num_steps=num_destinations)

print(f'Routes: {routes3}')

print(f'Best cost: {feasible_sampleset3.lowest().first.energy}')

Routes: [[0, 0, 0, 0, 0, 0, 0, 7, 2, 10, 13, 8, 5, 0, 0], [0, 4, 14, 6, 12, 15, 9, 11, 3, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
Best cost: 90.0


In [ ]:
val, route = lazy_sanity_check(num_destinations, distances=cost_matrix, num_vehicles=num_vehicles, max_distance=max_distance)
print(val)
print(route)
print(num_destinations)

### Compare to results with a reduced qubit representation

In [6]:
qubit_efficient_model = BoundedPathModel(num_locations=num_destinations, distances=cost_matrix, num_vehicles=num_vehicles, max_distance=max_distance)
qubit_efficient_model.build_constrained_model()
feasible_sampleset4 = qubit_efficient_model.run_constrained_model(token=TOKEN)


Feasible solution found.



In [9]:
# Consider the size of the problem
print(qubit_efficient_model.cqm)

Constrained quadratic model: 432 variables, 33 constraints, 13797 biases

Objective
  5*Binary('0_1_0') + 8*Binary('0_2_0') + 9*Binary('0_3_0') + 5*Binary('0_4_0') + 5*Binary('0_5_0') + 8*Binary('0_6_0') + 5*Binary('0_7_0') + 7*Binary('0_8_0') + 7*Binary('0_9_0') + 9*Binary('0_10_0') + 7*Binary('0_11_0') + 8*Binary('0_12_0') + 7*Binary('0_13_0') + 8*Binary('0_14_0') + 9*Binary('0_15_0') + 5*Binary('1_1_0') + 8*Binary('1_2_0') + 9*Binary('1_3_0') + 5*Binary('1_4_0') + 5*Binary('1_5_0') + 8*Binary('1_6_0') + 5*Binary('1_7_0') + 7*Binary('1_8_0') + 7*Binary('1_9_0') + 9*Binary('1_10_0') + 7*Binary('1_11_0') + 8*Binary('1_12_0') + 7*Binary('1_13_0') + 8*Binary('1_14_0') + 9*Binary('1_15_0') + 5*Binary('2_1_0') + 8*Binary('2_2_0') + 9*Binary('2_3_0') + 5*Binary('2_4_0') + 5*Binary('2_5_0') + 8*Binary('2_6_0') + 5*Binary('2_7_0') + 7*Binary('2_8_0') + 7*Binary('2_9_0') + 9*Binary('2_10_0') + 7*Binary('2_11_0') + 8*Binary('2_12_0') + 7*Binary('2_13_0') + 8*Binary('2_14_0') + 9*Binary('2_15_0'

In [ ]:
def parse_string(input_string):
    return list(map(int, input_string.split('_')))

def get_routes_from_sample(sample, num_vehicles, num_steps):
    """Builds a set of routes from the sample returned."""

    routes =  [[-1]*num_steps for _ in range(num_vehicles)]

    # Go through all entries
    for key, val in sample.items():
        vehicle, vertex, step = parse_string(key)
        if val == 1.0:
            routes[vehicle][step] = vertex

    # Clean up trailing and leading values (not optimized)
    # for route in routes:
    #     while route:
    #         if route[0] == 5:
    #             route.pop(0)
    #         else:
    #             break
    #     while route:
    #         if route[-1] == 5:
    #             route.pop(len(route) - 1)
    #         else:
    #             break
    
    return routes

In [18]:
lowest_energy_sample4 = feasible_sampleset4.lowest().first.sample

print(lowest_energy_sample4)

# print(feasible_sampleset4)

routes4 = get_routes_from_sample(lowest_energy_sample4, num_vehicles=num_vehicles, num_steps=num_destinations)

print(f'Routes: {routes4}')

print(f'Best cost: {feasible_sampleset4.lowest().first.energy}')

{'0_0_0': 1.0, '0_0_1': 1.0, '0_0_10': 0.0, '0_0_11': 0.0, '0_0_12': 0.0, '0_0_13': 1.0, '0_0_14': 0.0, '0_0_2': 1.0, '0_0_3': 1.0, '0_0_4': 1.0, '0_0_5': 0.0, '0_0_6': 0.0, '0_0_7': 0.0, '0_0_8': 0.0, '0_0_9': 0.0, '0_10_0': 0.0, '0_10_1': 0.0, '0_10_10': 0.0, '0_10_11': 0.0, '0_10_12': 0.0, '0_10_13': 0.0, '0_10_14': 0.0, '0_10_2': 0.0, '0_10_3': 0.0, '0_10_4': 0.0, '0_10_5': 0.0, '0_10_6': 0.0, '0_10_7': 0.0, '0_10_8': 0.0, '0_10_9': 1.0, '0_11_0': 0.0, '0_11_1': 0.0, '0_11_10': 0.0, '0_11_11': 0.0, '0_11_12': 0.0, '0_11_13': 0.0, '0_11_14': 0.0, '0_11_2': 0.0, '0_11_3': 0.0, '0_11_4': 0.0, '0_11_5': 0.0, '0_11_6': 1.0, '0_11_7': 0.0, '0_11_8': 0.0, '0_11_9': 0.0, '0_12_0': 0.0, '0_12_1': 0.0, '0_12_10': 0.0, '0_12_11': 1.0, '0_12_12': 0.0, '0_12_13': 0.0, '0_12_14': 0.0, '0_12_2': 0.0, '0_12_3': 0.0, '0_12_4': 0.0, '0_12_5': 0.0, '0_12_6': 0.0, '0_12_7': 0.0, '0_12_8': 0.0, '0_12_9': 0.0, '0_13_0': 0.0, '0_13_1': 0.0, '0_13_10': 0.0, '0_13_11': 0.0, '0_13_12': 0.0, '0_13_13': 0.0, 

NameError: name 'parse_string' is not defined